<b><font color="red" size="6">ch10. 데이터베이스 연동</font></b>

# 1절. SQLite 데이터 베이스 연결
- SQLite 데이터 베이스는 별도의 DBMS없이 SQL을 이용하여 DB 엑세스 할 수 있도록 만든 간단한 디스크 기반의 DB제공
- C 라이브러리
- SQLite는 프로토타입을 만들 때 사용
- [DB browser for SQLite](https://sqlitebrowser.org/dl/)
## 1.1 SQLite broswer 설치 및 sqlite3 패키지 load

In [1]:
import sqlite3
sqlite3.sqlite_version # sqlite3 패키지의 버전

'3.40.1'

In [2]:
import pandas as pd
pd.__version__

'1.5.3'

In [3]:
import numpy as np
np.__version__

'1.23.5'

## 1.2 데이터베이스 연결
- SQLite로 DB 연결시, DB파일이 있으면 연결, DB파일이 없으면 빈 DB 파일 생성

In [4]:
# # DB 연결
conn = sqlite3.connect('data/ch10_example.db')
conn

In [5]:
# 커서 객체 생성 커서는 SQL문 실행시키고, 결과데이터를 조회(select, 그외)하는데 사용하는 객체
cursor = conn.cursor()
cursor

In [14]:
cursor.execute('DROP TABLE MEMBER')

ProgrammingError: Cannot operate on a closed cursor.

In [12]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [14]:
cursor.execute("""
    INSERT INTO MEMBER VALUES ('홍길동', 20, 'h@h.com')
""")
print('수행결과 행수 :', cursor.rowcount)
sql = "INSERT INTO MEMBER VALUES ('신길동', 25, 's@s.com')"
cursor.execute(sql)
print('수행결과 행수 :', cursor.rowcount)
sql = "INSERT INTO MEMBER VALUES ('김길동', 30, 'k@k.com')"
cursor.execute(sql)
print('수행결과 행수 :', cursor.rowcount)

수행결과 행수 : 1
수행결과 행수 : 1
수행결과 행수 : 1


In [15]:
conn.commit() # 반대 : conn.rollback() DML문에서만 commit

In [19]:
cursor.execute("""
    SELECT * FROM MEMBER ORDER BY NAME
""")

-1


In [20]:
# insert, update, delete 문 실행 결과: cursor.rowcount
# select문 실행 결과를 받는 함수들
    ## fetchone() : 결과를 한행씩 받을 때 (튜플)
    ## fetchall() : 결과를 모두 받을 때 (튜플 list)
    ## fetchmany(n) : 결과를 n행 받을 때 (튜플 list)

In [21]:
print(cursor.fetchall())

[('김길동', 30, 'k@k.com'), ('신길동', 25, 's@s.com'), ('홍길동', 20, 'h@h.com'), ('홍길동', 20, 'h@h.com')]


In [22]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch할 수 없음

[]


In [11]:
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members # 튜플 리스트

[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('신길동', 25, 's@s.com'),
 ('김길동', 30, 'k@k.com'),
 ('구길동', 32, 'g@g.com'),
 ('바보', 999, 'babo@b.com')]

In [25]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('홍길동', 20, 'h@h.com')
('신길동', 25, 's@s.com')
('김길동', 30, 'k@k.com')


In [28]:
# 한줄씩 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
member_list = []
while True:
    member = cursor.fetchone() # 한줄 SQL문 수행 결과 가져오기
    if member is None:
        break
    member_list.append({'name':member[0], 'age':member[1], 'email':member[2]})

for member in member_list:
    print(member)

{'name': '홍길동', 'age': 20, 'email': 'h@h.com'}
{'name': '홍길동', 'age': 20, 'email': 'h@h.com'}
{'name': '신길동', 'age': 25, 'email': 's@s.com'}
{'name': '김길동', 'age': 30, 'email': 'k@k.com'}


In [29]:
# 최상위 n행 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'h@h.com')
('홍길동', 20, 'h@h.com')


In [30]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    def __str__(self):
        return 'name:{}, age:{}, email:{}'.format(self.name, self.age, self.email)
    def as_dic(self): # insert문 전송시 피로요
        return {'name':self.name, 'age':self.age, 'email':self.email}
def to_member(*row): # 튜플 데이터를 매개변수로 받아 Memeber형 객체로 return
    return Member(row[0], row[1], row[2])

In [33]:
dbreadmember = ('홍길동', 20, 'h@h.com')
m = to_member(*dbreadmember)
print(m)
print(m.as_dic())

name:홍길동, age:20, email:h@h.com
{'name': '홍길동', 'age': 20, 'email': 'h@h.com'}


In [17]:
# DB 검색 결과를 객체 list로
cursor.execute('SELECT * FROM MEMBER ORDER BY AGE DESC')
member_list = [] # sql문 수행 결과를 담을 객체 list
members = cursor.fetchall() # 튜플 list
# print(members)
for member in members:
    member_list.append(to_member(*member))

In [18]:
type(members[0]), type(member_list[0])

(tuple, __main__.Member)

In [37]:
for member in member_list:
    print(member)

name:김길동, age:30, email:k@k.com
name:신길동, age:25, email:s@s.com
name:홍길동, age:20, email:h@h.com
name:홍길동, age:20, email:h@h.com


In [12]:
cursor.close() # cursor.close()는 생략 가능
conn.close()

## 1.3 SQL구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우가 있음)
- named(추천)

In [6]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN('홍길동', '신길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('홍길동', 20, 'h@h.com'), ('신길동', 25, 's@s.com')]

In [7]:
# 파라미터 사용하기 : qmark 방법 이용
name1 = input('검색할 이름 1:')
name2 = input('검색할 이름 2:')
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?, ?)",(name1, name2))
cursor.fetchall()

검색할 이름 1:
검색할 이름 2:


[]

In [8]:
# 파라미터 사용하기 : qmark 방법 이용
name1 = input('검색할 이름 1:')
name2 = input('검색할 이름 2:')
names = [name1, name2]
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?, ?)",names)
cursor.fetchall()

검색할 이름 1:
검색할 이름 2:


[]

In [9]:
# 파라미터 사용하기 : named 방법 이용
name = input('검색할 이름 >')
cursor.execute("SELECT * FROM MEMBER WHERE NAME =:name",{'name':name})
members = cursor.fetchall()
if len(members):
    print(members)
else:
    print('조회하신 이름에 대한 정보가 없습니다')

검색할 이름 >
조회하신 이름에 대한 정보가 없습니다


In [10]:
# 파라미터 사용하기 : named 방법 이용
name1 = input('검색할 이름1 >')
name2 = input('검색할 이름2 >')
names = {'name1':name1, 'name2':name2}
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)",names)
cursor.fetchall()

검색할 이름1 >
검색할 이름2 >


[]

In [11]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    def __str__(self):
        return 'name:{}, age:{}, email:{}'.format(self.name, self.age, self.email)
    def as_dic(self): # insert문 전송시 피로요
        return {'name':self.name, 'age':self.age, 'email':self.email}
def to_member(*row): # 튜플 데이터를 매개변수로 받아 Memeber형 객체로 return
    return Member(row[0], row[1], row[2])

In [19]:
# member 테이블에 입력(사용자로부터 이름, 나이, 메일을 입력받아 insert)
while True:
    try:
        name = input('입력할 이름 >')
        age = int(input('입력할 나이를 정수로 입력하세요 >'))
        email = input('입력할 이메일 >')
        break
    except:
        print('정보를 정확히 다시 입력해주세요')
member_info = {'name':name, 'age':age, 'email':email}
cursor.execute(
    "INSERT INTO MEMBER VALUES (:name, :age, :email)",member_info
)
conn.commit()
if cursor.rowcount == 1:
    print('입력 성공')

입력할 이름 >바보
입력할 나이를 정수로 입력하세요 >999
입력할 이메일 >babo@b.com
입력 성공


In [28]:
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchall()

[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('신길동', 25, 's@s.com'),
 ('김길동', 30, 'k@k.com'),
 ('바보', 999, 'babo@b.com'),
 ('멍총이', 22, '총총'),
 ('0', 0, '0'),
 ('김찬밥', 52, '0'),
 ('ㅇ', 2, '0'),
 ('1.2', 2, '0'),
 ('2', 2, '0'),
 ('2', 2, '0'),
 ('3', 3, '0')]

In [13]:
# member 테이블에 입력(사용자로부터 이름, 나이, 메일을 입력받아 insert)
while True:
    try:
        name = input('입력할 이름 >')
        age = int(input('입력할 나이를 정수로 입력하세요 >'))
        email = input('입력할 이메일 >')
        break
    except:
        print('정보를 정확히 다시 입력해주세요')
cursor.execute(
    "INSERT INTO MEMBER VALUES (:name, :age, :email)",Member(name, age, email).as_dic()
)
conn.commit()
if cursor.rowcount == 1:
    print('입력 성공')

입력할 이름 >0
입력할 나이를 정수로 입력하세요 >0
입력할 이메일 >0
입력 성공


In [30]:
# member 테이블에 데이터를 n번 입력하고
# 이름에 0을 입력할 때까지 이름, 나이, 메일을 받아 insert
# 이름에 0을 입력하면 이때까지 입력한 데이터들을 출력하고 몇명을 입력했는지도 출력

member_list = []
while True:
    try:
        name = input('입력할 사용자의 이름을 입력하세요(종료하시려면 0을 입력하세요) > ')
        if name == '0':
            break
        age = int(input('입력할 사용자의 나이을 입력하세요 > '))
        email = input('입력할 사용자의 이메일을 입력하세요 > ')
        newMember = Member(name, age, email).as_dic()
        member_list.append(newMember)
        cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)",newMember)
        conn.commit()
        print(newMember, '님의 정보가 저장되었습니다.')
    except:
        print('정보를 정확히 다시 입력해주세요')
for member in member_list:
    print(newMember, '님의 정보가 저장되었습니다.')
print('사용자 {}명 입력 완료'.format(len(member_list)))

입력할 사용자의 이름을 입력하세요(종료하시려면 0을 입력하세요) > eiod
입력할 사용자의 나이을 입력하세요 > 22222
입력할 사용자의 이메일을 입력하세요 > ddddd
{'name': 'eiod', 'age': 22222, 'email': 'ddddd'} 님의 정보가 저장되었습니다.
입력할 사용자의 이름을 입력하세요(종료하시려면 0을 입력하세요) > 0
{'name': 'eiod', 'age': 22222, 'email': 'ddddd'} 님의 정보가 저장되었습니다.
사용자 1명 입력 완료


In [31]:
# 입력확인은 DB browser 나 python으로
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchall()

[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('신길동', 25, 's@s.com'),
 ('김길동', 30, 'k@k.com'),
 ('바보', 999, 'babo@b.com'),
 ('멍총이', 22, '총총'),
 ('0', 0, '0'),
 ('김찬밥', 52, '0'),
 ('ㅇ', 2, '0'),
 ('1.2', 2, '0'),
 ('2', 2, '0'),
 ('2', 2, '0'),
 ('3', 3, '0'),
 ('2', 3, '0'),
 ('eiod', 22222, 'ddddd')]

In [32]:
cursor.close()
conn.close()

# 2절. 오라클 데이터베이스 연결
- [cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/) 사용

In [33]:
import cx_Oracle

In [40]:
# conn 얻어오는 방법 1
oracle_dsn = cx_Oracle.makedsn(host='localhost', port=1521, sid='xe')
print(oracle_dsn)
conn = cx_Oracle.connect('dlow123', 'tmvjswl', dsn=oracle_dsn)
conn

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))


<cx_Oracle.Connection to dlow123@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [41]:
conn.close()

In [43]:
# conn 얻어오는 방법 2
conn = cx_Oracle.connect('dlow123','tmvjswl','localhost:1521/xe')

In [1]:
#cursor 객체 생성
cursor = conn.cursor()
sql = 'SELECT * FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()

NameError: name 'conn' is not defined

In [49]:
for e in emp:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [50]:
import pandas as pd
emp_df = pd.DataFrame(emp)
emp_df.head()

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [51]:
# select 문 수행한 필드 정보
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [53]:
result = []
for f in cursor.description:
    result.append(f[0])
result

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [56]:
[row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [57]:
emp_df.columns

RangeIndex(start=0, stop=8, step=1)

In [58]:
emp_df.columns = [row[0] for row in cursor.description]
emp_df.sample()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20


In [61]:
# 검색할 이름을 사용자에게 받아 해당 내용을 출력
ename = input('검색할 이름:')
sql = "SELECT * FROM EMP WHERE ENAME = UPPER(:name)"
cursor.execute(sql, {'name':ename})
emp = cursor.fetchall()
if len(emp):
    for e in emp:
        print(e)
else:
    print('입력하신 이름의 데이터는 없습니다')

검색할 이름:scott
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)


In [67]:
fieldnames = [description[0] for description in cursor.description]
print(fieldnames)
print(e)
for idx in range(len(e)):
    print('{}:{}'.format(fieldnames[idx], e[idx] if e[idx] is not None else ''), end='\t')
print()

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
EMPNO:7788	ENAME:SCOTT	JOB:ANALYST	MGR:7566	HIREDATE:1982-12-09 00:00:00	SAL:3000.0	COMM:	DEPTNO:20	


In [69]:
for fieldname, data in zip(fieldnames, e):
    print('{}:{}'.format(fieldname, data if data is not None else ''))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM:
DEPTNO:20


In [70]:
cursor.close()
conn.close()

# 3절. 연습문제

In [85]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    def __str__(self):
        return 'name:{}\nphone:{}\nemail:{}\nage:{}\ngrade:{}\netc:{}\n'.format(self.name, self.phone, self.email, self.age, self.grade, self.etc)
    def as_dict(self):
        return {'name':self.name, 'phone':self.phone, 'email':self.email, 'age':self.age, 'grade':self.grade, 'etc':self.etc}
def to_member(*row):
    return Member(row[0],row[1],row[2],row[3],row[4],row[5])

In [73]:
def insert_member():
    cursor = conn.cursor()
    name = input('등록하실분의 이름을 입력하세요 > ')
    phone = input('등록하실분의 전화번호을 입력하세요 > ')
    email = input('등록하실분의 이메일을 입력하세요 > ')
    while True:
        try:
            age = int(input('등록하실분의 나이을 정수로 입력하세요 > '))
            grade = int(input('등록하실분의 등급을 정수로 입력하세요 > '))
            break
        except:
            print('정수로 다시 입력해주세요')
    etc = input('등록하실분의 기타사항을 입력하세요 > ')
    cursor.execute('INSERT INTO MEMBER VALUES (:name, :phone, :email, :age, :grade, :etc)', Member(name, phone, email, age, grade, etc).as_dict())
    conn.commit()
    print('입력 성공!')
    cursor.close()

In [83]:
def select_all():
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM MEMBER')
    members = cursor.fetchall()
    for member in members:
        print(to_member(*member))
    cursor.close()

In [88]:
def select_name():
    cursor = conn.cursor()
    name = input('검색하실분의 이름을 작성해 주세요')
    cursor.execute('SELECT * FROM MEMBER WHERE NAME = UPPER(:name)',{'name':name})
    members = cursor.fetchall()
    for member in members:
        print(to_member(*member))
    cursor.close()

In [111]:
def delete_email():
    cursor = conn.cursor()
    email = input('삭제하실분의 이메일을 작성해 주세요')
    cursor.execute('DELETE FROM MEMBER WHERE EMAIL = :email',{'email':email})
    conn.commit()
    if cursor.rowcount >= 1:
        print('삭제완료되었습니다')
    cursor.close()

In [97]:
def save_csv():
    import csv
    cursor = conn.cursor()
    save_member_list = []
    cursor.execute('SELECT * FROM MEMBER')
    members = cursor.fetchall()
    for member in members:
        save_member_list.append(to_member(*member).as_dict())
    fieldnames = list(save_member_list[0].keys())
    filename = input('저장할 CSV 파일 이름은 ?')
    try :
        with open('data/{}.csv'.format(filename), 'w', newline='', encoding='UTF-8') as f:
            dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
            dict_writer.writeheader()
            dict_writer.writerows(save_member_list)
    except Exception as e:
        print('데이터 내보내기 예외 :', e)
    cursor.close()

In [103]:
def close_sql():
    conn.close()
    print('프로그램이 종료되었습니다')

In [101]:
def main():
    while True:
        print("1.입력", "2.전체 조회", "3.이름 찾기", "4.메일 삭제", "5.CSV 내보내기", "0.종료",sep=" | ")
        try:
            menu = int(input("메뉴 선택: "))
        except:
            print("유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.")
        if menu==1:
            insert_member()
        elif menu==2:
            select_all()
        elif menu==3:
            select_name()
        elif menu==4:
            delete_email()
        elif menu==5:
            save_csv()
        elif menu==0:
            close_sql()
            break

In [110]:
if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data/ch10_data.db')
    main()

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 4
삭제하실분의 이메일을 작성해 주세요k@k.com
삭제완료되었습니다
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 2
name:홍길동
phone:010-9999-9999
email:h@h.com
age:20
grade:3
etc:바보

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 0
프로그램이 종료되었습니다


```
1. 파이썬의데이터베이스 연동에대해잘 못 설명한 것은?
답 : (4) commit()을 호출하지 않고 데이터베이스 연결 객체를 close() 하면 변경사항은 자동으로 저장된다.

2. 파이썬의Cursor 객체에대해잘 못 설명한 것은?
답 : (2) 커서객체의execute() 메소드는결과set으로 반환한다
```